# Initial Setup

Run this notebook to set up the catalogs and schema, as well as create the fuel types CSV file. Once done:

* upload your fuel data from https://www.developer.fuel-finder.service.gov.uk/access-latest-fuelprices to the `prices` directory in the `bronze.petrol_prices.csv` volume
* upload your postcode data to the `postcode` directory in the `bronze.petrol_prices.csv` volume

Then, import the `petrol_prices_etl_pipeline` as a Pipeline, and the Json file in `dashboards` as a Dashboard.

Once you've done that, run the metric view notebook in `metric_views`. You can now run your pipeline, and when completed, you should have a dashboard available to you!

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS bronze;
CREATE SCHEMA IF NOT EXISTS bronze.petrol_prices;
CREATE VOLUME IF NOT EXISTS bronze.petrol_prices.csv;

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS silver;
CREATE SCHEMA IF NOT EXISTS silver.petrol_prices;

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS gold;
CREATE SCHEMA IF NOT EXISTS gold.petrol_prices;

In [0]:
import os
os.makedirs("/Volumes/bronze/petrol_prices/csv/fuel_types/", exist_ok=True)
os.makedirs("/Volumes/bronze/petrol_prices/csv/postcode/", exist_ok=True)
os.makedirs("/Volumes/bronze/petrol_prices/csv/prices/", exist_ok=True)

In [0]:
from pyspark.sql.types import StructType, StructField, StringType

fuel_types = spark.createDataFrame(
    [
        ["E5", "Super Unleaded"],
        ["E10", "Unleaded"],
        ["B7S", "Diesel"],
        ["B7P", "Premium Diesel"],
        ["B10", "Biodiesel"],
        ["HVO", "Hydrogen"]
    ],
    schema=StructType(
        [
            StructField("fuel_code", StringType(), True),
            StructField("fuel_name", StringType(), True)
        ]
    )
)

fuel_types.write.mode("overwrite").format("csv").option("header", "true").save("/Volumes/bronze/petrol_prices/csv/fuel_types/fuel_types.csv")